In [5]:
!pip install kagglehub

In [10]:
import kagglehub

# downloading bread basket dataset
path = kagglehub.dataset_download("mittalvasu95/the-bread-basket")

print("Path to dataset files:", path)

Path to dataset files: /Users/beketbarlykov/.cache/kagglehub/datasets/mittalvasu95/the-bread-basket/versions/1


In [11]:
!mv /Users/beketbarlykov/.cache/kagglehub/datasets/mittalvasu95/the-bread-basket/versions/1 ./data_raw/bread_basket.csv

In [1]:
import pandas as pd

transactions_df = pd.read_csv('./data_raw/bread_basket/bread_basket.csv')
transactions_df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [2]:
start = transactions_df.index.min()
end = transactions_df.index.max()

In [3]:
transactions_df.tail()

,Transaction,Item,date_time,period_day,weekday_weekend
20502,9682,Coffee,09-04-2017 14:32,afternoon,weekend
20503,9682,Tea,09-04-2017 14:32,afternoon,weekend
20504,9683,Coffee,09-04-2017 14:57,afternoon,weekend
20505,9683,Pastry,09-04-2017 14:57,afternoon,weekend
20506,9684,Smoothies,09-04-2017 15:04,afternoon,weekend


In [6]:
import requests
import os

RAPIDAPI_KEY = os.environ.get("RAPID_API_KEY")
if not RAPIDAPI_KEY:
    raise EnvironmentError("Please set the RAPIDAPI_KEY environment variable.")
# Edinburgh coordinates
edinburgh_lat = 55.9533
edinburgh_long = -3.1883

# Common headers
headers = {
    "x-rapidapi-host": "meteostat.p.rapidapi.com",
    "x-rapidapi-key": RAPIDAPI_KEY
}

# -------------------------
# 1. Find nearest station(s)
# -------------------------
url = "https://meteostat.p.rapidapi.com/stations/nearby"
params = {
    "lat": edinburgh_lat,
    "lon": edinburgh_long,
    "limit": 5
}

In [8]:
res = requests.get(url, headers=headers, params=params)
if res.status_code != 200:
    raise Exception(f"Error fetching stations: {res.text}")

In [10]:
stations_data = res.json()
print("Nearby stations:", stations_data)

Nearby stations: {'meta': {'generated': '2025-08-13 09:58:42'}, 'data': [{'id': '03160', 'name': {'en': 'Edinburgh Airport'}, 'distance': 10073.7}, {'id': '03171', 'name': {'en': 'Leuchars'}, 'distance': 51793.4}, {'id': '03163', 'name': {'en': 'Dundee / Invergowrie / Balmerino / Wormit'}, 'distance': 56241.2}, {'id': 'EGPT0', 'name': {'en': 'Perth / Saint Martins'}, 'distance': 56322.1}, {'id': '03158', 'name': {'en': 'Charterhall'}, 'distance': 57629.8}]}


In [11]:
station_id = stations_data['data'][1]['id']
print("Using station:", station_id)

Using station: 03171


In [18]:
# Getting weather hourly data
url = 'https://meteostat.p.rapidapi.com/stations/hourly'
params = {
    "station": station_id,
    "start": "2016-10-30",   # YYYY-MM-DD
    "end": "2016-11-29",     # YYYY-MM-DD
    "tz": "Europe/London"
}

In [19]:
res = requests.get(url, headers=headers, params=params)
if res.status_code != 200:
    raise Exception(f"Error getting hourly data: {res.text}")

In [20]:
data = res.json()

In [22]:
print(data)

{'meta': {'generated': '2025-08-13 10:04:14'}, 'data': [{'time': '2016-10-30 00:00:00', 'temp': 11.7, 'dwpt': 10.4, 'rhum': 92.0, 'prcp': None, 'snow': None, 'wdir': 250.0, 'wspd': 14.8, 'wpgt': None, 'pres': 1029.3, 'tsun': None, 'coco': None}, {'time': '2016-10-30 01:00:00', 'temp': 11.6, 'dwpt': 9.8, 'rhum': 89.0, 'prcp': None, 'snow': None, 'wdir': 250.0, 'wspd': 9.4, 'wpgt': None, 'pres': 1029.2, 'tsun': None, 'coco': None}, {'time': '2016-10-30 01:00:00', 'temp': 11.6, 'dwpt': 9.5, 'rhum': 87.0, 'prcp': None, 'snow': None, 'wdir': 240.0, 'wspd': 18.4, 'wpgt': None, 'pres': 1028.6, 'tsun': None, 'coco': None}, {'time': '2016-10-30 02:00:00', 'temp': 11.4, 'dwpt': 9.1, 'rhum': 86.0, 'prcp': None, 'snow': None, 'wdir': 240.0, 'wspd': 16.6, 'wpgt': None, 'pres': 1028.4, 'tsun': None, 'coco': None}, {'time': '2016-10-30 03:00:00', 'temp': 10.9, 'dwpt': 8.3, 'rhum': 84.0, 'prcp': None, 'snow': None, 'wdir': 240.0, 'wspd': 16.6, 'wpgt': None, 'pres': 1028.2, 'tsun': None, 'coco': None},

In [25]:
from datetime import datetime, timedelta
# Date range
start_date = datetime(2016, 10, 30)
end_date = datetime(2017, 4, 9)
interval_days = 30

In [30]:
chunks_dir = "data_external/edinburgh_weather_chunks"
os.makedirs(chunks_dir, exist_ok=True)

In [31]:
chunk_num = 1
current_start = start_date
all_chunks = []  # list to hold DataFrames for merging

In [32]:
while current_start <= end_date:
    current_end = min(current_start + timedelta(days=interval_days - 1), end_date)
    params = {
        "station": station_id,
        "start": current_start.strftime("%Y-%m-%d"),
        "end": current_end.strftime("%Y-%m-%d"),
        "tz": "Europe/London"
    }
    
    print(f"Fetching chunk {chunk_num}: {params['start']} → {params['end']}")
    
    response_hourly = requests.get(url, headers=headers, params=params)
    if response_hourly.status_code != 200:
        raise Exception(f"Error fetching hourly data: {response_hourly.text}")
    
    data = response_hourly.json().get("data", [])
    
    if data:
        df_chunk = pd.DataFrame(data)
        chunk_path = os.path.join(chunks_dir, f"weather_chunk_{chunk_num}.csv")
        df_chunk.to_csv(chunk_path, index=False)
        print(f"Saved {len(df_chunk)} rows to {chunk_path}")
        
        all_chunks.append(df_chunk)
    else:
        print(f"No data for chunk {chunk_num}")
    
    chunk_num += 1
    current_start = current_end + timedelta(days=1)

Fetching chunk 1: 2016-10-30 → 2016-11-28
Saved 721 rows to data_external/edinburgh_weather_chunks/weather_chunk_1.csv
Fetching chunk 2: 2016-11-29 → 2016-12-28
Saved 720 rows to data_external/edinburgh_weather_chunks/weather_chunk_2.csv
Fetching chunk 3: 2016-12-29 → 2017-01-27
Saved 720 rows to data_external/edinburgh_weather_chunks/weather_chunk_3.csv
Fetching chunk 4: 2017-01-28 → 2017-02-26
Saved 720 rows to data_external/edinburgh_weather_chunks/weather_chunk_4.csv
Fetching chunk 5: 2017-02-27 → 2017-03-28
Saved 719 rows to data_external/edinburgh_weather_chunks/weather_chunk_5.csv
Fetching chunk 6: 2017-03-29 → 2017-04-09
Saved 288 rows to data_external/edinburgh_weather_chunks/weather_chunk_6.csv


In [33]:
if all_chunks:
    df_all = pd.concat(all_chunks, ignore_index=True)
    merged_path = "data_external/edinburgh_hourly_weather_full.csv"
    df_all.to_csv(merged_path, index=False)
    print(f"Merged CSV saved to {merged_path} with {len(df_all)} rows")
else:
    print("No data collected; merged file not created.")

Merged CSV saved to data_external/edinburgh_hourly_weather_full.csv with 3888 rows


In [34]:
holidays_url = 'https://www.gov.uk/bank-holidays.json'
holidays_data = requests.get(holidays_url).json()
holidays_data

{'england-and-wales': {'division': 'england-and-wales',
  'events': [{'title': 'New Year’s Day',
    'date': '2024-01-01',
    'notes': '',
    'bunting': True},
   {'title': 'Good Friday',
    'date': '2024-03-29',
    'notes': '',
    'bunting': False},
   {'title': 'Easter Monday',
    'date': '2024-04-01',
    'notes': '',
    'bunting': True},
   {'title': 'Early May bank holiday',
    'date': '2024-05-06',
    'notes': '',
    'bunting': True},
   {'title': 'Spring bank holiday',
    'date': '2024-05-27',
    'notes': '',
    'bunting': True},
   {'title': 'Summer bank holiday',
    'date': '2024-08-26',
    'notes': '',
    'bunting': True},
   {'title': 'Christmas Day',
    'date': '2024-12-25',
    'notes': '',
    'bunting': True},
   {'title': 'Boxing Day', 'date': '2024-12-26', 'notes': '', 'bunting': True},
   {'title': 'New Year’s Day',
    'date': '2025-01-01',
    'notes': '',
    'bunting': True},
   {'title': 'Good Friday',
    'date': '2025-04-18',
    'notes': '',
 

In [35]:
holidays_df = pd.DataFrame(holidays_data)
holidays_df.head()

,england-and-wales,scotland,northern-ireland
division,england-and-wales,scotland,northern-ireland
events,"[{'title': 'New Year’s Day', 'date': '2024-01-...","[{'title': 'New Year’s Day', 'date': '2024-01-...","[{'title': 'New Year’s Day', 'date': '2024-01-..."


In [37]:
holidays_df['scotland']['events']

[{'title': 'New Year’s Day',
  'date': '2024-01-01',
  'notes': '',
  'bunting': True},
 {'title': '2nd January', 'date': '2024-01-02', 'notes': '', 'bunting': True},
 {'title': 'Good Friday', 'date': '2024-03-29', 'notes': '', 'bunting': False},
 {'title': 'Early May bank holiday',
  'date': '2024-05-06',
  'notes': '',
  'bunting': True},
 {'title': 'Spring bank holiday',
  'date': '2024-05-27',
  'notes': '',
  'bunting': True},
 {'title': 'Summer bank holiday',
  'date': '2024-08-05',
  'notes': '',
  'bunting': True},
 {'title': 'St Andrew’s Day',
  'date': '2024-12-02',
  'notes': 'Substitute day',
  'bunting': True},
 {'title': 'Christmas Day',
  'date': '2024-12-25',
  'notes': '',
  'bunting': True},
 {'title': 'Boxing Day', 'date': '2024-12-26', 'notes': '', 'bunting': True},
 {'title': 'New Year’s Day',
  'date': '2025-01-01',
  'notes': '',
  'bunting': True},
 {'title': '2nd January', 'date': '2025-01-02', 'notes': '', 'bunting': True},
 {'title': 'Good Friday', 'date': '2

In [38]:
scotland_holidays= holidays_df['scotland']['events']

In [41]:
sco_hol_df = pd.DataFrame(scotland_holidays)

In [43]:
sco_hol_df.to_csv("./data_external/uk_bank_holidays.csv", index=False)

In [44]:
transactions_df.head()

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [46]:
df_all.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2016-10-30 00:00:00,11.7,10.4,92.0,None,None,250.0,14.8,None,1029.3,None,NaN
1,2016-10-30 01:00:00,11.6,9.8,89.0,None,None,250.0,9.4,None,1029.2,None,NaN
2,2016-10-30 01:00:00,11.6,9.5,87.0,None,None,240.0,18.4,None,1028.6,None,NaN
3,2016-10-30 02:00:00,11.4,9.1,86.0,None,None,240.0,16.6,None,1028.4,None,NaN
4,2016-10-30 03:00:00,10.9,8.3,84.0,None,None,240.0,16.6,None,1028.2,None,NaN


In [47]:
sco_hol_df.head()

,title,date,notes,bunting
0,New Year’s Day,2024-01-01,,True
1,2nd January,2024-01-02,,True
2,Good Friday,2024-03-29,,False
3,Early May bank holiday,2024-05-06,,True
4,Spring bank holiday,2024-05-27,,True


In [48]:
weather_df = df_all

In [49]:
weather_df = weather_df.drop(columns=["wdir", "wpgt", "tsun", "coco"], errors="ignore")
weather_df["time"] = pd.to_datetime(weather_df["time"])
weather_df["date"] = weather_df["time"].dt.date
weather_df["hour"] = weather_df["time"].dt.hour

In [50]:
transactions_df["date_time"] = pd.to_datetime(transactions_df["date_time"], format="%d-%m-%Y %H:%M")
transactions_df["date"] = transactions_df["date_time"].dt.date
transactions_df["hour"] = transactions_df["date_time"].dt.hour

In [51]:
# 3. Merge transactions with weather on date + hour
merged_df = pd.merge(
    transactions_df,
    weather_df,
    how="left",
    left_on=["date", "hour"],
    right_on=["date", "hour"]
)


In [65]:
# Ensure both date columns are datetime
merged_df["date"] = pd.to_datetime(merged_df["date"])
sco_hol_df["date"] = pd.to_datetime(sco_hol_df["date"])

# Extract month-day string for matching
merged_df["month_day"] = merged_df["date"].dt.strftime("%m-%d")
sco_hol_df["month_day"] = sco_hol_df["date"].dt.strftime("%m-%d")

# Create a set of holiday month-days for fast lookup
holiday_set = set(sco_hol_df["month_day"])

# Flag holidays
merged_df["is_holiday"] = merged_df["month_day"].isin(holiday_set).astype(int)

# Drop helper column if not needed
merged_df.drop(columns=["month_day"], inplace=True)

In [66]:
merged_df.count()

Transaction        20507
Item               20507
date_time          20507
period_day         20507
weekday_weekend    20507
date               20507
hour               20507
time               20507
temp               20499
dwpt               20499
rhum               20499
prcp                   0
snow                   0
wspd               20499
pres               20499
is_holiday         20507
dtype: int64

In [67]:
merged_df.head()

,Transaction,Item,date_time,period_day,weekday_weekend,date,hour,time,temp,dwpt,rhum,prcp,snow,wspd,pres,is_holiday
0,1,Bread,2016-10-30 09:58:00,morning,weekend,2016-10-30,9,2016-10-30 09:00:00,10.7,9.5,92.0,None,None,14.8,1027.9,0
1,2,Scandinavian,2016-10-30 10:05:00,morning,weekend,2016-10-30,10,2016-10-30 10:00:00,11.2,9.6,90.0,None,None,14.8,1027.9,0
2,2,Scandinavian,2016-10-30 10:05:00,morning,weekend,2016-10-30,10,2016-10-30 10:00:00,11.2,9.6,90.0,None,None,14.8,1027.9,0
3,3,Hot chocolate,2016-10-30 10:07:00,morning,weekend,2016-10-30,10,2016-10-30 10:00:00,11.2,9.6,90.0,None,None,14.8,1027.9,0
4,3,Jam,2016-10-30 10:07:00,morning,weekend,2016-10-30,10,2016-10-30 10:00:00,11.2,9.6,90.0,None,None,14.8,1027.9,0


In [68]:
merged_df.drop(columns=["date", "hour", "time"], inplace=True)

In [71]:
merged_df['prcp'] = merged_df['prcp'].fillna(0)
merged_df['snow'] = merged_df['snow'].fillna(0)

/var/folders/cj/g3bq0_2s5698155pxxd5t14m0000gn/T/ipykernel_1177/1731922488.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['prcp'] = merged_df['prcp'].fillna(0)
/var/folders/cj/g3bq0_2s5698155pxxd5t14m0000gn/T/ipykernel_1177/1731922488.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['snow'] = merged_df['snow'].fillna(0)


In [72]:
merged_df.head()

,Transaction,Item,date_time,period_day,weekday_weekend,temp,dwpt,rhum,prcp,snow,wspd,pres,is_holiday
0,1,Bread,2016-10-30 09:58:00,morning,weekend,10.7,9.5,92.0,0,0,14.8,1027.9,0
1,2,Scandinavian,2016-10-30 10:05:00,morning,weekend,11.2,9.6,90.0,0,0,14.8,1027.9,0
2,2,Scandinavian,2016-10-30 10:05:00,morning,weekend,11.2,9.6,90.0,0,0,14.8,1027.9,0
3,3,Hot chocolate,2016-10-30 10:07:00,morning,weekend,11.2,9.6,90.0,0,0,14.8,1027.9,0
4,3,Jam,2016-10-30 10:07:00,morning,weekend,11.2,9.6,90.0,0,0,14.8,1027.9,0


In [118]:
# Example DataFrame (use your merged_df)
df = merged_df.copy()

# 1. Create mapping dictionary for items
item_to_id = {item: idx for idx, item in enumerate(df["Item"].unique(), start=1)}

# Optional: Save dictionary for later use
# pd.Series(item_to_id).to_csv("item_id_mapping.csv")

# 2. Map items to IDs
df["item_id"] = df["Item"].map(item_to_id)

# 3. Convert weekday/weekend to binary
df["weekday_weekend"] = df["weekday_weekend"].map({"weekday": 0, "weekend": 1})

# 4. Drop 'period_day'
df.drop(columns=["period_day"], inplace=True)

# # 5. Group transactions
# df_grouped = (
#     df.groupby(
#         ["Transaction", "date_time", "weekday_weekend", "temp", "dwpt", "rhum", "prcp", "snow", "wspd", "pres", "is_holiday"],
#         as_index=False
#     )["item_id"]
#     .apply(list)
# )

In [115]:
print(item_to_id)       # Shows the dictionary

{'Bread': 1, 'Scandinavian': 2, 'Hot chocolate': 3, 'Jam': 4, 'Cookies': 5, 'Muffin': 6, 'Coffee': 7, 'Pastry': 8, 'Medialuna': 9, 'Tea': 10, 'Tartine': 11, 'Basket': 12, 'Mineral water': 13, 'Farm House': 14, 'Fudge': 15, 'Juice': 16, "Ella's Kitchen Pouches": 17, 'Victorian Sponge': 18, 'Frittata': 19, 'Hearty & Seasonal': 20, 'Soup': 21, 'Pick and Mix Bowls': 22, 'Smoothies': 23, 'Cake': 24, 'Mighty Protein': 25, 'Chicken sand': 26, 'Coke': 27, 'My-5 Fruit Shoot': 28, 'Focaccia': 29, 'Sandwich': 30, 'Alfajores': 31, 'Eggs': 32, 'Brownie': 33, 'Dulce de Leche': 34, 'Honey': 35, 'The BART': 36, 'Granola': 37, 'Fairy Doors': 38, 'Empanadas': 39, 'Keeping It Local': 40, 'Art Tray': 41, 'Bowl Nic Pitt': 42, 'Bread Pudding': 43, 'Adjustment': 44, 'Truffles': 45, 'Chimichurri Oil': 46, 'Bacon': 47, 'Spread': 48, 'Kids biscuit': 49, 'Siblings': 50, 'Caramel bites': 51, 'Jammie Dodgers': 52, 'Tiffin': 53, 'Olum & polenta': 54, 'Polenta': 55, 'The Nomad': 56, 'Hack the stack': 57, 'Bakewell':

In [121]:
final_df = df.rename(columns={"Transaction": "transaction_id"})
final_df.drop(columns=['Item'], inplace=True)

In [122]:
print(final_df.head()) # Shows grouped data

   transaction_id           date_time  weekday_weekend  temp  dwpt  rhum  \
0               1 2016-10-30 09:58:00                1  10.7   9.5  92.0   
1               2 2016-10-30 10:05:00                1  11.2   9.6  90.0   
2               2 2016-10-30 10:05:00                1  11.2   9.6  90.0   
3               3 2016-10-30 10:07:00                1  11.2   9.6  90.0   
4               3 2016-10-30 10:07:00                1  11.2   9.6  90.0   

   prcp  snow  wspd    pres  is_holiday  item_id  
0     0     0  14.8  1027.9           0        1  
1     0     0  14.8  1027.9           0        2  
2     0     0  14.8  1027.9           0        2  
3     0     0  14.8  1027.9           0        3  
4     0     0  14.8  1027.9           0        4  


In [123]:
import json

final_df.to_csv('./data_processed/data.csv', index=False)

In [124]:
mapping_df = pd.DataFrame(list(item_to_id.items()), columns=["item", "item_id"])

# Save to CSV
mapping_df.to_csv("./data_processed/item_id_mapping.csv", index=False)

In [81]:
df_grouped.columns

Index(['Transaction', 'date_time', 'weekday_weekend', 'temp', 'dwpt', 'rhum',
       'prcp', 'snow', 'wspd', 'pres', 'is_holiday', 'item_id'],
      dtype='object')